# Imports

In [60]:
import pandas as pd

In [61]:
import xml.etree.ElementTree as et

In [62]:
import os
import sys

In [63]:
import matplotlib.pyplot as plt

In [64]:
#import argparse

# Define and parse command-line arguments
#parser = argparse.ArgumentParser(description='Process some integers.')
#parser.add_argument('--seed', type=int, help='Seed value')
#parser.add_argument('--traffic_scale', type=float, help='Traffic scale value')
#args = parser.parse_args()

# Functions

In [65]:
def get_last_folder(path):
    # Normalize the path to handle different separators and remove trailing separator
    normalized_path = os.path.normpath(path)
    # Split the path into components
    folders = normalized_path.split(os.sep)
    # Get the last folder
    last_folder = folders[-1]
    return last_folder

In [66]:
price_per_wh = 70 / 1000

In [67]:
def get_price(d_energy):
    return d_energy * price_per_wh

In [68]:
def transform_xml(xml_doc):
    attr = xml_doc.attrib
    for xml in xml_doc.iter('vehicle'):
        _dict = attr.copy()
        _dict.update(xml.attrib)
        
        yield _dict


In [69]:
def calc_elevation_up(group):
    z_diff = pd.to_numeric(group['z']).diff()

    # Filter out negative differences (upward movement)
    up = z_diff.apply(lambda x: x if x > 0 else 0)

    # Sum the positive differences to get the total upward movement
    total_up = up.sum()
    return total_up

In [70]:
def calc_elevation_down(group):
    z_diff = pd.to_numeric(group['z']).diff()

    # Filter out negative differences (upward movement)
    down = z_diff.apply(lambda x: x if x < 0 else 0)

    # Sum the positive differences to get the total upward movement
    total_down = down.sum()
    return total_down

In [71]:
def transform_xml_tripinfo(xml_doc):
    attr = xml_doc.attrib
    for xml in xml_doc.iter('tripinfo'):
        _dict = attr.copy()
        _dict.update(xml.attrib)
        
        yield _dict

In [72]:
def transform_xml_stops(xml_doc):
    for route in xml_doc.iter('route'):
        route_dict = route.attrib.copy()
        stops = []
        
        for stop in route.findall('stop'):
            stop_dict = stop.attrib.copy()
            stops.append(stop_dict)
        
        route_dict['stops'] = stops
        
        yield route_dict

In [73]:
def get_group_by_id(list_of_dfs, desired_id):
    for df in list_of_dfs:
        if desired_id in df['id'].values:
            return df[df['id'] == desired_id]
    raise ValueError(f"ID '{desired_id}' not found in any dataframe.")

# XML to df

In [74]:
base_folder = "C:\\Users\\Admin\\Sumo\\nap_gellert_b"

In [75]:
file_path = os.path.join(base_folder, "emission.out.xml")
emission_output = et.parse(file_path)

transform = transform_xml(emission_output.getroot())
emission_output_list = list(transform)

emission_output_df = pd.DataFrame(emission_output_list)
emission_output_df = emission_output_df.drop(emission_output_df.columns[0], axis=1)

emission_output_df.shape

(34230, 20)

In [76]:
file_path = os.path.join(base_folder, "Battery.out.xml")

battery_output = et.parse(file_path)
battery_output_root = battery_output.getroot()

transform = transform_xml(battery_output_root)
battery_output_list = list(transform)

battery_output_pd = pd.DataFrame(battery_output_list)

battery_output_pd = battery_output_pd.drop(battery_output_pd.columns[0], axis=1)
battery_output_pd

,id,energyConsumed,totalEnergyConsumed,totalEnergyRegenerated,actualBatteryCapacity,maximumBatteryCapacity,chargingStationId,energyCharged,energyChargedInTransit,energyChargedStopped,speed,acceleration,x,y,lane,posOnLane,timeStopped
0,C767841140.0,0.0000,0.0000,0.0000,1500.0000,3000.0000,NULL,0.0000,0.0000,0.0000,0.0000,0.0000,4054.5714,1655.1857,296910360#1_0,12.1000,0
1,C767841140.0,6.2090,6.2090,0.0000,1493.7910,3000.0000,NULL,0.0000,0.0000,0.0000,1.0000,1.0000,4053.6881,1654.7200,296910360#1_0,13.1000,0
2,C767841140.0,15.1682,21.3771,0.0000,1478.6229,3000.0000,NULL,0.0000,0.0000,0.0000,2.0000,1.0000,4051.9213,1653.7887,296910360#1_0,15.1000,0
3,C767841140.0,24.1416,45.5187,0.0000,1454.4813,3000.0000,NULL,0.0000,0.0000,0.0000,3.0000,1.0000,4049.2713,1652.3918,296910360#1_0,18.1000,0
4,C767841140.0,33.1278,78.6465,0.0000,1421.3535,3000.0000,NULL,0.0000,0.0000,0.0000,4.0000,1.0000,4045.7379,1650.5292,296910360#1_0,22.1000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34225,C782574680.0,0.0309,7163.4740,2232.8023,23.3951,3000.0000,NULL,0.0000,0.0000,0.0000,0.0000,0.0000,4578.4906,2123.8592,20831647#1_0,18.9660,136
34226,C782451142.0,7.8440,7270.1218,1855.7089,164.7384,3000.0000,NULL,0.0000,0.0000,0.0000,6.0198,0.0000,467.6256,1306.9310,24677999#0_0,35.8236,0
34227,C782574680.0,0.0309,7163.5049,2232.8023,23.3643,3000.0000,NULL,0.0000,0.0000,0.0000,0.0000,0.0000,4578.4906,2123.8592,20831647#1_0,18.9660,137
34228,C782451142.0,7.8440,7277.9659,1855.7089,156.8944,3000.0000,NULL,0.0000,0.0000,0.0000,6.0198,0.0000,472.1135,1310.9423,24677999#0_0,41.8434,0


In [77]:
file_path = os.path.join(base_folder, "tripinfo.xml")

b_tripinfo_output = et.parse(file_path)
b_tripinfo_output_root = b_tripinfo_output.getroot()

transform = transform_xml_tripinfo(b_tripinfo_output_root)
b_tripinfo_output_list = list(transform)

b_tripinfo_output_pd = pd.DataFrame(b_tripinfo_output_list)

b_tripinfo_output_pd = b_tripinfo_output_pd.drop(b_tripinfo_output_pd.columns[0], axis=1)
b_tripinfo_output_pd

,id,depart,departLane,departPos,departSpeed,departDelay,arrival,arrivalLane,arrivalPos,arrivalSpeed,...,routeLength,waitingTime,waitingCount,stopTime,timeLoss,rerouteNo,devices,vType,speedFactor,vaporized
0,C767841140.0,1970.00,296910360#1_0,12.10,0.00,0.00,2689.00,54727328_0,73.00,8.00,...,1012.78,576.00,4,20.00,627.70,0,tripinfo_C767841140.0 emissions_C767841140.0 b...,ElectricBus,1.06,
1,C76784379.0,10910.00,296910360#1_0,12.10,0.00,0.00,11639.00,54727328_0,73.00,8.00,...,1012.78,586.00,4,20.00,637.45,0,tripinfo_C76784379.0 emissions_C76784379.0 bat...,ElectricBus,1.05,
2,C6305325.0,12950.00,34572881#1_0,12.10,0.00,0.00,13513.00,-50219636#1_0,56.74,9.00,...,1702.65,346.00,3,30.00,402.49,0,tripinfo_C6305325.0 emissions_C6305325.0 batte...,ElectricBus,0.94,
3,C77632401.0,16070.00,34572881#1_0,12.10,0.00,0.00,16695.00,-50219636#1_0,56.74,9.00,...,1702.65,397.00,4,30.00,456.04,0,tripinfo_C77632401.0 emissions_C77632401.0 bat...,ElectricBus,0.87,
4,C7769412.0,18890.00,292524113#0_0,12.10,0.00,0.00,19279.00,600260151_0,113.30,10.00,...,1636.63,137.00,2,64.00,216.15,0,tripinfo_C7769412.0 emissions_C7769412.0 batte...,ElectricBus,1.09,
5,C78257148.0,23030.00,548875276#0_0,12.10,0.00,0.00,24151.00,-581325727_0,332.48,10.00,...,3311.51,576.00,6,95.00,705.22,0,tripinfo_C78257148.0 emissions_C78257148.0 bat...,ElectricBus,0.98,
6,C6359838.0,24410.00,34572881#1_0,12.10,0.00,0.00,24861.00,34572878#1_0,48.32,1.00,...,575.27,347.00,2,20.00,375.25,0,tripinfo_C6359838.0 emissions_C6359838.0 batte...,ElectricBus,1.09,
7,C76796152.0,25130.00,53864729#4_0,12.10,0.00,0.00,25195.00,53864729#4_0,445.46,1.00,...,433.36,1.00,1,20.00,16.97,0,tripinfo_C76796152.0 emissions_C76796152.0 bat...,ElectricBus,1.12,
8,C7810534.0,25610.00,581325727_0,12.10,0.00,0.00,25862.00,25440988#0_0,85.69,8.05,...,2169.76,0.00,0,20.00,20.95,0,tripinfo_C7810534.0 emissions_C7810534.0 batte...,ElectricBus,0.97,
9,C76647202.0,26330.00,894166783_0,4.14,0.00,0.00,27215.00,311385575_0,48.95,0.00,...,1055.47,716.00,7,20.00,780.15,0,tripinfo_C76647202.0 emissions_C76647202.0 bat...,ElectricBus,1.02,teleport


In [78]:
file_path = os.path.join(base_folder, "gtfs_pt_vehicles.add.xml")
stops = et.parse(file_path)

transform = transform_xml_stops(stops.getroot())
stops_list = list(transform)

stops_pd = pd.DataFrame(stops_list)

In [79]:
file_path = os.path.join(base_folder, "gtfs_pt_vehicles.add.xml")
vehicles = et.parse(file_path)

transform = transform_xml(vehicles.getroot())
vehicles_list = list(transform)

vehicles_pd = pd.DataFrame(vehicles_list)
vehicles_pd = vehicles_pd.drop(vehicles_pd.columns[0], axis=1)
#vehicles_pd

## Grouping by id

In [80]:
grouped_df = battery_output_pd.groupby('id')

list_of_dfs = [group_data for _, group_data in grouped_df]
#C74509106 = list_of_dfs[0]
#C74509124 = list_of_dfs[1]
#C74509135 = list_of_dfs[2]
list_of_dfs

[                 id energyConsumed totalEnergyConsumed totalEnergyRegenerated  \
 14243  C622152254.0         0.0000              0.0000                 0.0000   
 14245  C622152254.0         4.4999              4.4999                 0.0000   
 14247  C622152254.0        11.8490             16.3488                 0.0000   
 14249  C622152254.0        19.3771             35.7259                 0.0000   
 14251  C622152254.0        27.1621             62.8880                 0.0000   
 ...             ...            ...                 ...                    ...   
 15228  C622152254.0         0.0309           3617.0301              1488.8997   
 15230  C622152254.0         2.4277           3619.4578              1488.8997   
 15232  C622152254.0         6.8502           3626.3080              1488.8997   
 15234  C622152254.0        10.6362           3636.9443              1488.8997   
 15236  C622152254.0        15.1207           3652.0649              1488.8997   
 
       actualB

In [81]:
grouped_emission_df = emission_output_df.groupby('id')

list_of_emission_dfs = [group_data for _, group_data in grouped_emission_df]
list_of_emission_dfs



[                 id                             eclass   CO2    CO    HC  \
 14243  C622152254.0  HBEFA4/UBus_Electric_Std_gt15-18t  0.00  0.00  0.00   
 14245  C622152254.0  HBEFA4/UBus_Electric_Std_gt15-18t  0.00  0.00  0.00   
 14247  C622152254.0  HBEFA4/UBus_Electric_Std_gt15-18t  0.00  0.00  0.00   
 14249  C622152254.0  HBEFA4/UBus_Electric_Std_gt15-18t  0.00  0.00  0.00   
 14251  C622152254.0  HBEFA4/UBus_Electric_Std_gt15-18t  0.00  0.00  0.00   
 ...             ...                                ...   ...   ...   ...   
 15228  C622152254.0  HBEFA4/UBus_Electric_Std_gt15-18t  0.00  0.00  0.00   
 15230  C622152254.0  HBEFA4/UBus_Electric_Std_gt15-18t  0.00  0.00  0.00   
 15232  C622152254.0  HBEFA4/UBus_Electric_Std_gt15-18t  0.00  0.00  0.00   
 15234  C622152254.0  HBEFA4/UBus_Electric_Std_gt15-18t  0.00  0.00  0.00   
 15236  C622152254.0  HBEFA4/UBus_Electric_Std_gt15-18t  0.00  0.00  0.00   
 
         NOx   PMx  fuel electricity  noise       route         type waiti

In [84]:
for group_id, group_data in grouped_df:
    avg_speed = group_data['speed'].astype(float).mean()
    
    energy = float(group_data['totalEnergyConsumed'].iloc[-1])-float(group_data['totalEnergyRegenerated'].iloc[-1])
    print(get_price(energy))

151.42156400000002
173.253549
40.683237000000005
75.86750500000001
91.41086499999999
820.742874
807.900338
602.8243200000002
572.0688050000001
621.4495350000001
489.0588710000001
299.62840600000004
180.074384
146.016759
126.09844100000002
118.74181899999999
167.79779100000005
131.619019
118.112092
149.15831000000006
41.78706700000001
41.808668999999995
46.640783
43.706887
-2.3288510000000033
-2.3288510000000033
49.73495100000001
-2.3288510000000033
215.366991
2350.603024
1476.304711
218.20432899999997
151.22279200000003
124.324732
122.84839000000004
459.8852650000001
296.9775690000001
471.2127280000001
185.63173300000005
319.058278
316.1026750000001
190.47837200000004
379.55799
1089.4394840000002
1473.735375
522.5909570000001
526.7682910000001
345.15134500000005


In [83]:

results = []
for group_id, group_data in grouped_df:
    avg_speed = group_data['speed'].astype(float).mean()
    
    energy = get_price(float(group_data['totalEnergyConsumed'].iloc[-1])-float(group_data['totalEnergyRegenerated'].iloc[-1]))
    print(float(group_data['totalEnergyConsumed'].iloc[-1])-float(group_data['totalEnergyRegenerated'].iloc[-1]))
    
    time_loss = b_tripinfo_output_pd.loc[b_tripinfo_output_pd['id'] == group_id, 'timeLoss'].values[0]
    route_length = b_tripinfo_output_pd.loc[b_tripinfo_output_pd['id'] == group_id, 'routeLength'].values[0]
    
    route = vehicles_pd.loc[vehicles_pd['id'] == group_id, 'route'].values[0]
    count_stops = stops_pd[stops_pd['id'] == route]['stops'].apply(len).sum()
    
    z_up = calc_elevation_up(get_group_by_id(list_of_emission_dfs, group_id))
    z_down = calc_elevation_down(get_group_by_id(list_of_emission_dfs, group_id))
    
    # Store the results in a dictionary
    group_result = {
        'id': group_id,
        'avgSpeed': avg_speed,
        'battery': energy,
        'timeloss': time_loss,
        'routeLength': route_length,
        'numOfStops': count_stops,
        'up': z_up,
        'down': z_down
    }
    
    # Append the dictionary to the results list
    results.append(group_result)

# Convert the results list to a DataFrame
result_df = pd.DataFrame(results)
#print(result_df)


2163.1652
2475.0507
581.1891
1083.8215
1305.8694999999998
11724.8982
11541.4334
8611.776000000002
8172.411500000001
8877.8505
6986.555300000001
4280.4058
2572.4912
2085.9537
1801.4063
1696.3116999999997
2397.1113000000005
1880.2716999999998
1687.3156
2130.8330000000005
596.9581000000001
597.2666999999999
666.2968999999999
624.3841
-33.269300000000044
-33.269300000000044
710.4993000000001
-33.269300000000044
3076.6713
33580.0432
21090.0673
3117.2046999999993
2160.3256
1776.0675999999999
1754.9770000000003
6569.789500000001
4242.536700000001
6731.6104000000005
2651.8819000000003
4557.975399999999
4515.7525000000005
2721.1196000000004
5422.257


IndexError: index 0 is out of bounds for axis 0 with size 0

# CSV

In [ ]:
tableBattery = pd.read_csv('batteryData.csv', delimiter=';')

tableBattery.shape

(65, 11)

## settings

In [ ]:
locSetting = get_last_folder(base_folder)
seedSetting = 'fgh'
trafficScaleSetting = 'asd'

#routeLengthSetting = :)
#numOfStopsSetting = :)
#tripIdSetting = :)
#avgSpeedSetting = :)
#timelossSetting = :)
#eleupSetting = 
#eledownSetting = 
#emissionSetting = :)

## Df to csv

In [ ]:
for index, row in result_df.iterrows():
    row_data = {
        'loc': locSetting,
        'tripId': row['id'],
        'seed': seedSetting,
        'avgSpeed': row['avgSpeed'],
        'timeloss': row['timeloss'],
        'route_length': row['routeLength'],
        'elevation_up': row['up'],
        'elevation_down': row['down'],
        'trafficScale': trafficScaleSetting,
        'numOfStops': row['numOfStops'],
        'emission': row['battery']
    }
    temp_df = pd.DataFrame([row_data])
    #any empty or all-NA columns in tableBattery are excluded before concatenating the DataFrames
    tableBattery = tableBattery.dropna(axis=1, how='all')

    tableBattery = pd.concat([tableBattery, temp_df], ignore_index=True)


'id': group_id,
        'avgSpeed': avg_speed,
        'batteryCapacity': energy,
        'timeloss': time_loss,
        'routeLength': route_length,
        'stops': count_stops

In [ ]:
tableBattery.to_csv('batteryData.csv', index=False, sep=';')
tableBattery

,loc,tripId,seed,avgSpeed,timeloss,route_length,elevation_up,elevation_down,trafficScale,numOfStops,emission
0,nap_gellert_b,C6359838.0,fgh,3.600735,109.62,673.45,8.72,-10.81,asd,2,4.485516
1,nap_gellert_b,C734723628.0,fgh,5.563921,138.52,3647.97,105.48,-68.31,asd,12,41.629497
2,nap_gellert_b,C745091034.0,fgh,2.790037,980.82,4075.61,22.54,-79.52,asd,11,10.840263
3,nap_gellert_b,C74509178.0,fgh,1.558993,2138.63,3973.37,81.46,-24.90,asd,11,46.163117
4,nap_gellert_b,C74509688.0,fgh,1.574278,1905.35,3994.44,81.32,-24.76,asd,11,46.106459
...,...,...,...,...,...,...,...,...,...,...,...
119,nap_gellert_b,C78245617.0,fgh,3.679731,1007.18,8072.59,172.65,-218.99,asd,19,1097.776813
120,nap_gellert_b,C78245945.0,fgh,4.521554,852.81,8690.53,240.24,-194.20,asd,18,1488.215589
121,nap_gellert_b,C78257148.0,fgh,2.735513,751.02,3311.51,95.64,-98.63,asd,4,531.160378
122,nap_gellert_b,C782571889.0,fgh,5.269959,122.40,2724.58,87.39,-83.98,asd,4,531.428016
